In [1]:
import sys
import os
import collections
from collections import defaultdict
import numpy as np
import pandas as pd
from scipy import stats
import re

In [2]:
df = pd.read_csv('clean_sample_50k.csv')
# df = pd.read_csv('clean_sample_30k.csv')
# df = pd.read_csv('scrape_1_1999.csv')
df.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
df.head()

,University,Major,Program,Citizenship,Semester,Year,Q_Score,V_Score,AWA_Score,GPA_Score,Status
0,University Of Minnesota,Pure Mathematics,PhD,American,Fall,2022,168,164,4.0,3.8,Accepted
1,Deccan College Of Engineering And Technology,Electrical And Computer Engineering,Masters,International,Fall,2022,165,150,4.5,3.7,Accepted
2,University Of Southern California,Applied Economics,Masters,American,Fall,2022,149,157,4.0,3.5,Accepted
3,Columbia University,Applied Mathematics,Masters,International,Fall,2022,170,164,5.0,0.0,Accepted
4,University Of Tennessee Knoxville,Computer Engineering,PhD,International,Fall,2022,162,145,4.0,3.7,Accepted


In [20]:
df = df[df['Program'].str.contains("PhD")==False]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16164 entries, 1 to 27806
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   University   16164 non-null  object 
 1   Major        16164 non-null  object 
 2   Program      16164 non-null  object 
 3   Citizenship  16164 non-null  object 
 4   Semester     16164 non-null  object 
 5   Year         16164 non-null  int64  
 6   Q_Score      16164 non-null  int64  
 7   V_Score      16164 non-null  int64  
 8   AWA_Score    16164 non-null  float64
 9   GPA_Score    16164 non-null  float64
 10  Status       16164 non-null  object 
dtypes: float64(2), int64(3), object(6)
memory usage: 1.5+ MB


In [21]:
df.shape

(16164, 11)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16164 entries, 1 to 27806
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   University   16164 non-null  object 
 1   Major        16164 non-null  object 
 2   Program      16164 non-null  object 
 3   Citizenship  16164 non-null  object 
 4   Semester     16164 non-null  object 
 5   Year         16164 non-null  int64  
 6   Q_Score      16164 non-null  int64  
 7   V_Score      16164 non-null  int64  
 8   AWA_Score    16164 non-null  float64
 9   GPA_Score    16164 non-null  float64
 10  Status       16164 non-null  object 
dtypes: float64(2), int64(3), object(6)
memory usage: 1.5+ MB


In [23]:
# Creating dataframe for only those entries which has status = accepted
df = df[df['Status'] == 'Accepted']
df.reset_index(drop = True, inplace = True)
df.head()

,University,Major,Program,Citizenship,Semester,Year,Q_Score,V_Score,AWA_Score,GPA_Score,Status
0,Deccan College Of Engineering And Technology,Electrical And Computer Engineering,Masters,International,Fall,2022,165,150,4.5,3.7,Accepted
1,University Of Southern California,Applied Economics,Masters,American,Fall,2022,149,157,4.0,3.5,Accepted
2,Columbia University,Applied Mathematics,Masters,International,Fall,2022,170,164,5.0,0.0,Accepted
3,Texas A&M Kingsville,Communication Disorders,Masters,American,Fall,2022,143,155,4.0,3.5,Accepted
4,UCL,Economics,Masters,International,Fall,2022,165,157,4.0,3.7,Accepted


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16164 entries, 0 to 16163
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   University   16164 non-null  object 
 1   Major        16164 non-null  object 
 2   Program      16164 non-null  object 
 3   Citizenship  16164 non-null  object 
 4   Semester     16164 non-null  object 
 5   Year         16164 non-null  int64  
 6   Q_Score      16164 non-null  int64  
 7   V_Score      16164 non-null  int64  
 8   AWA_Score    16164 non-null  float64
 9   GPA_Score    16164 non-null  float64
 10  Status       16164 non-null  object 
dtypes: float64(2), int64(3), object(6)
memory usage: 1.4+ MB


In [37]:
processed_data = df[['Q_Score', 'V_Score', 'AWA_Score', 'GPA_Score', 'University']]
processed_data

,Q_Score,V_Score,AWA_Score,GPA_Score,University
0,165,150,4.5,3.7,Deccan College Of Engineering And Technology
1,149,157,4.0,3.5,University Of Southern California
2,170,164,5.0,0.0,Columbia University
3,143,155,4.0,3.5,Texas A&M Kingsville
4,165,157,4.0,3.7,UCL
...,...,...,...,...,...
16159,170,162,4.5,3.2,NYU Tandon School Of Engineering
16160,170,154,3.5,3.5,Politecnico Di Milano (Polimi)
16161,170,154,3.5,3.5,TU Delft Netherlands
16162,166,161,4.5,3.8,UCL(University College London)


In [38]:
import math
from sklearn import neighbors, datasets
from numpy.random import permutation
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support

In [39]:
random_indices = permutation(processed_data.index)
test_cutoff = math.floor(len(processed_data)/5)
# print(test_cutoff)

test = processed_data.loc[random_indices[1:test_cutoff]]
train = processed_data.loc[random_indices[test_cutoff:]]

train_output_data = train['University']
# print("train Output data", train_output_data)

train_input_data = train
train_input_data = train_input_data.drop('University',1)
# print("train input data", train_input_data)

test_output_data = test['University']
# print("test Output data", test_output_data)

test_input_data = test
test_input_data = test_input_data.drop('University',1)
# print("test input data", test_input_data)

In [40]:

def euclideanDistance(data1, data2, length):
    distance = 0
    for x in range(length):
        distance += np.square(data1[x] - data2[x])
    return np.sqrt(distance)

def knn(trainingSet, testInstance, k):
    print(k)
    distances = {}
    sort = {}
    length = testInstance.shape[1]

    for x in range(len(trainingSet)):

        dist = euclideanDistance(testInstance, trainingSet.iloc[x], length)
        
        distances[x] = dist[0]

    sorted_d = sorted(distances.items(), key=lambda x: x[1])


    neighbors = []

    for x in range(k):
        neighbors.append(sorted_d[x][0])

    classVotes = {}

    for x in range(len(neighbors)):
        response = trainingSet.iloc[neighbors[x]][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1

    sortedVotes = sorted(classVotes.items(), key=lambda x: x[1], reverse=True)
    
    return(sortedVotes, neighbors)


In [41]:
testSet = [[155, 155, 4.0, 3]]
test = pd.DataFrame(testSet)
test.head()

,0,1,2,3
0,155,155,4.0,3


In [42]:

k = 15

result,neigh= knn(processed_data, test, k)


list1 = []
# list2 = []
for i in result:
    list1.append(i[0])
#     list2.append(i[1])
for i in list1:
    print(i)
    

15
 John Jay College Of Criminal Justice
 American International College
 Marymount University
 Miami University (OH)
 SUNY Buffalo
 University Of New Mexico
 Florida International University
 University Of Arizona
 University Of Miami
 University Of Illinois Chicago (UIC)
University Of Texas At Austin
 San Diego State (SDSU)
 Curry School Of Education - University Of Virginia
 Loyola Marlyand
 St. Cloud State University


In [ ]:
# train test split
# in built knn
# it should recommmend the names of the univ. by passing in the scores
# accuracy of your model
